In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_squared_error as MSE

In [139]:
rest = pd.read_csv('/Volumes/Database/Research/ozone-budget/result_df_UKESM1_3_0_NN_width_64_dropout_0.1_90.csv')
data = pd.read_csv('/Volumes/Database/Research/C7 Multivariate analysis/UKESM1-0-LL_diag_2014(1217).csv')

In [140]:
rest['tas'] = data['tas']
rest['pan'] = data['pan']
rest['photo1d'] = data['photo1d']

rest['o3_mole'] = rest['o3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no_mole'] = rest['no']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no2_mole'] = rest['no2']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['oh_mole'] = rest['oh']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['h2o_mole'] = rest['h2o']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['hno3_mole'] = rest['hno3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['pan_mole'] = rest['pan']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['ho2_mole'] = rest['HO₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['ch3o2_mole'] = rest['CH₃O₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['o1d_mole'] = rest['O¹D']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['k_ho2_ho2'] = 2.2*10**(-13)*np.exp(600/rest['tas'])
rest['k_oh_no2'] = 6.5*10**(-11)
rest['k_no_ho2'] = data['k_no_ho2']
rest['k_no_ch3o2'] = data['k_no_ch3o2']
rest['k_o1d_h2o'] = data['k_o1d_h2o']
rest['k_o3_ho2'] = data['k_o3_ho2']
rest['k_o3_oh'] = data['k_o3_oh']

In [155]:
rest['P1_HO2_NO'] = rest['k_no_ho2']*rest['no_mole']*rest['ho2_mole']
rest['P2_CH3O2_NO'] = rest['k_no_ch3o2']*rest['no_mole']*rest['ch3o2_mole']

rest['P3_O1D_H2O'] = rest['o1d_mole']*rest['h2o_mole']*rest['k_o1d_h2o']
rest['P4_OH_O3'] = rest['k_o3_oh']*rest['oh_mole']*rest['o3_mole']
rest['P5_HO2_O3'] = rest['k_o3_ho2']*rest['ho2_mole']*rest['o3_mole']

In [156]:
rest['P1'] = rest['P1_HO2_NO']/rest['PO₃̂']
rest['P2'] = rest['P2_CH3O2_NO']/rest['PO₃̂']

rest['P3'] = rest['P3_O1D_H2O']/rest['LO₃̂']
rest['P4'] = rest['P4_OH_O3']/rest['LO₃̂']
rest['P5'] = rest['P5_HO2_O3']/rest['LO₃̂']

In [179]:
rest['HO2_OH'] = rest['ho2_mole']/rest['oh_mole']
rest['alpha'] = rest['P3_O1D_H2O']/rest['o3_mole']/rest['photo1d']

In [180]:
rest['alpha'].describe()

count    9.720000e+05
mean              inf
std               NaN
min      6.984042e-06
25%      6.848949e-02
50%      1.197735e-01
75%      1.607564e-01
max               inf
Name: alpha, dtype: float64

In [151]:
country = pd.read_csv('/Users/csuen/Desktop/PhD/Maps/country_province_state_2*2.csv')
rest = rest.merge(country, how = 'left', left_on = ['longitude','latitude'], right_on = ['longitude','latitude'])
rest = rest.drop(['province','state'], axis=1)
rest_land = rest.dropna()

In [143]:
rest_land['PO₃̂'].mean(), rest_land['Pt_HO2_NO'].mean(), rest_land['Pt_CH3O2_NO'].mean(), rest_land['Pt_RO2_NO'].mean()

(2771131.4303185223,
 1401355.8334046982,
 1331463.2983078114,
 38312.298606030454)

In [144]:
rest_land['LO₃̂'].mean(), rest_land['Pt_O1D_H2O'].mean(), rest_land['Pt_HO2_O3'].mean(), rest_land['Pt_OH_O3'].mean(), rest['Pt_VOCs_O3'].mean()

(688458.8364923124,
 573685.8740445137,
 2325.715233444886,
 67343.61033307547,
 33347.619734845735)

In [145]:
rest_land['Pt_VOCs_O3'].describe()

count    3.221400e+05
mean     4.510364e+04
std      1.156712e+05
min     -1.204887e+05
25%     -8.248697e+03
50%     -2.613205e+02
75%      3.949216e+04
max      2.268468e+06
Name: Pt_VOCs_O3, dtype: float64

In [138]:
rest['P4'] = rest['k_o3_oh']*rest['o3_mole']*rest['oh_mole']
rest['P4'].describe()

count    9.720000e+05
mean     6.405802e+04
std      9.152843e+04
min      2.661861e-03
25%      3.639479e+03
50%      2.636375e+04
75%      9.023354e+04
max      1.091792e+06
Name: P4, dtype: float64

In [146]:
rest['P1'] = rest['k_no_ho2']*rest['ho2_mole']*rest['no_mole']
rest['P1'].describe()

count    9.720000e+05
mean     6.106327e+05
std      2.219049e+06
min      1.828845e-09
25%      1.272309e+04
50%      7.788514e+04
75%      3.289972e+05
max      1.965647e+08
Name: P1, dtype: float64

In [147]:
rest['P5'] = rest['k_o3_ho2']*rest['ho2_mole']*rest['o3_mole']
rest['P5'].describe()

count    972000.000000
mean       4432.852045
std        4785.875438
min           0.000001
25%         498.736601
50%        2677.768324
75%        7388.049031
max       65214.393034
Name: P5, dtype: float64

In [150]:
rest['HO2_OH'] = rest['ho2_mole']/rest['oh_mole']
rest['HO2_OH'].describe()

count    9.720000e+05
mean     7.077818e+03
std      3.649433e+04
min      1.558228e-02
25%      2.533803e+02
50%      5.097629e+02
75%      1.146024e+03
max      2.736999e+06
Name: HO2_OH, dtype: float64

In [153]:
rest_land['HO2_OH'].describe()

count    322140.000000
mean      15619.443490
std       54547.733046
min           0.015582
25%          80.742548
50%         322.773522
75%        1075.948238
max      934847.242150
Name: HO2_OH, dtype: float64